<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict outdoor equipment purchase with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistence to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the Plotly package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist the model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Upload **GoSales Transactions** data as a data asset in IBM Data Science Experience.
-  Make sure that you are using a Spark 2.0 kernel.

### Create the GoSales Transactions Data Asset  

The GOSales data is a freely available data set on the Data Science Experience home page.

1.  Go to the [GoSales Transactions for Naive Bayes Model](https://apsportal.ibm.com/exchange/public/entry/view/8044492073eb964f46597b4be06ff5ea) data card on the Data Science Experience Community.
2.  Click the link icon.
4.  Select the link, copy it by pressing Ctrl+C, and then, click **Close**.
5.  In the cell in the next section, replace the **link_to_data** variable value with the link.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then run the *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install wget --user <BR>

In [5]:
import wget

link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = wget.download(link_to_data)

print filename

GoSales_Tx_NaiveBayes.csv


The csv file `GoSales_Tx_NaiveBayes.csv` is available on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [7]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)
df_data.take(5)

[Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'M', AGE=27, MARITAL_STATUS=u'Single', PROFESSION=u'Professional'),
 Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'F', AGE=39, MARITAL_STATUS=u'Married', PROFESSION=u'Other'),
 Row(PRODUCT_LINE=u'Mountaineering Equipment', GENDER=u'F', AGE=39, MARITAL_STATUS=u'Married', PROFESSION=u'Other'),
 Row(PRODUCT_LINE=u'Personal Accessories', GENDER=u'F', AGE=56, MARITAL_STATUS=u'Unspecified', PROFESSION=u'Hospitality'),
 Row(PRODUCT_LINE=u'Golf Equipment', GENDER=u'M', AGE=45, MARITAL_STATUS=u'Married', PROFESSION=u'Retired')]

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [8]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [9]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [10]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

Split your data into the train, test, and predict data sets:

In [11]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


The data has been split into three data sets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set is used for model evaluation and to test the assumptions of the model.
-  The predict data set is used for predictions.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

Import the Apache® Spark machine learning packages that you'll need in the subsequent steps:

In [13]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

Convert all the string fields to numeric ones by using the StringIndexer transformer:

In [14]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

Create a feature vector by combining all features together:

In [15]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define the estimators you want to use for classification. Random Forest is used in the following example.

In [16]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, convert indexed labels back to original labels:

In [17]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Now build the pipeline. A pipeline consists of transformers and an estimator.

In [18]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Train your Random Forest model by using your pipeline with the train_data set.

In [19]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [20]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. Evaluate the model with the test_data set.

In [21]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.589319
Test Error = 0.410681


You can now tune your model to achieve better accuracy. For simplicity, the tuning section is omitted from this example.

<a id="persistence"></a>
## 4. Persist the model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using Python client libraries.

First, you must import client libraries.

In [22]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

Put your authentication information from your instance of the Watson Machine Learning service on Bluemix in the next cell. You can find your information on the **Service Credentials** tab of your service instance in Bluemix.

In [23]:
service_path = 'https://ibm-watson-ml.stage1.mybluemix.net'
username = '827b8bbc-c0b7-4994-b3cf-0cb45371bad7'
password = '030528d4-5a3e-4d4c-9258-5d553513be6f'

Authorize the repository client:

In [24]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

Create the model artifact (abstraction layer).

In [25]:
model_artifact = MLRepositoryArtifact(model_rf, training_data=train_data, name="Product Line Prediction Model")

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save the pipeline and model

Save pipeline and model artifacts to your Watson Machine Learning instance:

In [26]:
saved_model = ml_repository_client.models.save(model_artifact)

Get the saved model metadata from Watson Machine Learning. Use the *meta.available_props()* method to get the list of available props.

In [27]:
saved_model.meta.available_props()

['inputDataSchema',
 'modelVersionHref',
 'pipelineVersionHref',
 'trainingDataRef',
 'pipelineType',
 'creationTime',
 'lastUpdated',
 'label',
 'authorEmail',
 'trainingDataSchema',
 'authorName',
 'version',
 'modelType',
 'runtime']

In [28]:
print "modelType: " + saved_model.meta.prop("modelType")
print "trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema"))
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")
print "label: " + saved_model.meta.prop("label")

modelType: sparkml-model-2.0
trainingDataSchema: {u'fields': [{u'nullable': True, u'type': u'string', u'name': u'PRODUCT_LINE', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'GENDER', u'metadata': {}}, {u'nullable': True, u'type': u'integer', u'name': u'AGE', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'MARITAL_STATUS', u'metadata': {}}, {u'nullable': True, u'type': u'string', u'name': u'PROFESSION', u'metadata': {}}], u'type': u'struct'}
creationTime: 2017-03-22 10:39:23.126000+00:00
modelVersionHref: https://ibm-watson-ml.stage1.mybluemix.net/v2/artifacts/models/e3250d67-04c6-4788-8f62-6ea8d8806464/versions/51841d33-b644-4acb-a4c0-0cafa6bd67c2
label: PRODUCT_LINE


**Tip**: **modelVersionHref** is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load the model

Load the saved model from the specified instance of Watson Machine Learning:

In [29]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [30]:
print str(loadedModelArtifact.name)

Product Line Prediction Model


As you can see the name is correct. You have learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with the Plotly package.

### 5.1: Make a local prediction using previously loaded model and test data

Score the predict_data data set:

In [31]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [27]:
predictions.show(5)

+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS| PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+-----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 18|        Single|      Other|  0.0|          0.0|      1.0|              1.0|[1.0,18.0,1.0,0.0]|[5.54119162527029...|[0.27705958126351...|       1.0|Personal Accessories|
|Camping Equipment|     F| 18|        Single|     Retail|  0.0|          7.0|      1.0|              1.0|[1.0,18.0,1.0,7.0]|[1.69052858961399...|[0.08452642948069...|       1.0|Personal Accessories|
|Camp

Tabulate a count to see which product line is the most popular:

In [32]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

+--------------------+-----+
|      predictedLabel|count|
+--------------------+-----+
|   Camping Equipment|  717|
|      Golf Equipment|   61|
|Mountaineering Eq...|   75|
|Personal Accessories|  363|
+--------------------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install plotly --user <BR>
!pip install cufflinks --user

Import Plotly and other required packages.

In [33]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go
init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [34]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [35]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()

product_data = [go.Pie(
            labels=cumulative_stats.index,
            values=cumulative_stats['GENDER'],
    )]

product_layout = go.Layout(
    title='Predicted product line client interest distribution',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [36]:
age_data = [go.Bar(
            y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"],
            x=cumulative_stats.index
            
    )]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(
        title = "Product Line",
        showline=False,),
    yaxis=dict(
        title = "Mean AGE",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Watson Machine Learning API Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API, generate an access token:

In [90]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v2/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create an online scoring endpoint

Now you can create an online scoring endpoint. Execute the following sample code that uses the **modelVersionHref** value to create the scoring endpoint to the Bluemix repository.

In [93]:
endpoint_online = service_path + "/v2/online/deployments/"
header_online = {'Content-Type': 'application/json', 'Authorization': mltoken}
payload_online = {"artifactVersionHref": saved_model.meta.prop("modelVersionHref"), "name": "Product Line Prediction"}

response_online = requests.post(endpoint_online, json=payload_online, headers=header_online)

print response_online
print response_online.text

scoring_href = json.loads(response_online.text).get('entity').get('scoringHref')
print scoring_href

<Response [201]>
{"metadata":{"guid":"971","href":"https://ibm-watson-ml.stage1.mybluemix.net/v2/online/deployments/971","createdAt":"2017-03-20T10:44:39.869Z","modifiedAt":"2017-03-20T10:44:39.869Z"},"entity":{"scoringHref":"https://ibm-watson-ml.stage1.mybluemix.net/32768/v2/scoring/971"}}
https://ibm-watson-ml.stage1.mybluemix.net/32768/v2/scoring/971


Send (PUT) new scoring records (new data) for which you would like to get predictions:

In [95]:
payload_scoring = {"record":["M", 23, "Single", "Student"]}
response_scoring = requests.put(scoring_href, json=payload_scoring, headers=header_online)

print response_scoring.text

{"result":{"PROFESSION_IX":6.0,"GENDER_IX":0.0,"MARITAL_STATUS_IX":1.0,"GENDER":"M","features":{"values":[0.0,23.0,1.0,6.0]},"predictedLabel":"Personal Accessories","prediction":1.0,"AGE":23,"MARITAL_STATUS":"Single","rawPrediction":{"values":[5.600716147152702,6.482458372136143,6.048004170730676,0.20929155492307386,1.6595297550574055]},"PROFESSION":"Student","probability":{"values":[0.2800358073576351,0.32412291860680714,0.3024002085365338,0.010464577746153694,0.08297648775287028]}}}


The prediction is that a 23-year-old male student is interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0).

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://apsportal.ibm.com/docs/content/getting-started/get-started.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.